In [2]:
#importing necessary libraries
import os
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
load_dotenv()

True

In [6]:
parser = LlamaParse(
    api_key=os.getenv("LLAMA_CLOUD_API_KEY"),
    result_type="markdown"  # "markdown" and "text" are available
)
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(input_files=['test_parsing/deeplearningbook.org_contents_part_basics.html.pdf'], file_extractor=file_extractor).load_data()
print(documents)

ValidationError: 1 validation error for LlamaParse
api_key
  Value error, The API key is required. [type=value_error, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error